# DESAFIO DO TITANIC

A proposta do desafio é prever quais passageiros sobreviveram ao naufrágio do Titanic, a partir de um conjunto de dados dos passageiros.

In [ ]:
# importa bibliotecas gerais

import warnings
warnings.filterwarnings('ignore')

# importa bibliotecas de preparação dos dados

import re
import numpy as np
import pandas as pd

# importa bibliotecas de visualização dos dados

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# carrega os dados

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# guarda PassengerId para fácil acesso

PassengerId = test['PassengerId']

## Análise exploratória dos dados de treino

In [ ]:
# informações gerais sobre o conjunto de dados de treino

train.info()

In [ ]:
# mostra as cinco primeiras linhas do conjunto de dados de treino

train.head()

In [ ]:
# mostra as cinco últimas linhas do conjunto de dados

train.tail()

In [ ]:
# visão geral da distribuição estatística dos dados de treino

train.describe()

In [ ]:
# histograma dos dados numéricos de treino

train.hist(figsize=(10,8));

In [ ]:
# histograma de idade dos passageiros do conjunto de dados de treino

plt.figure(figsize=(8,4))
fig = sns.distplot(train['Age'], color="darkgreen")
fig.set_xlabel("Idade",size=15)
fig.set_ylabel("Densidade dos Passageiros",size=15)
plt.title('Distribuição de Idade dos Passageiros',size = 20)
plt.show()

# média de idade dos passageiros do conjunto de dados de treino

print('Média de idade dos passageiros: ',train['Age'].mean())

In [ ]:
# histograma de tarifas dos passageiros do conjunto de dados de treino

plt.figure(figsize=(8,4))
fig = sns.distplot(train['Fare'], color="darkred")
fig.set_xlabel("Tarifa",size=15)
fig.set_ylabel("Densidade dos Passageiros",size=15)
plt.title('Distribuição de Tarifas dos Passageiros',size = 20)
plt.show()

# média de tarifa dos passageiros do conjunto de dados de treino

print('Média de tarifa dos passageiros: ',train['Fare'].mean())

In [ ]:
# plota gráficos para Survived x Sex, Pclass e Embarked

fig, (axis1, axis2, axis3) = plt.subplots(1,3, figsize=(12,4))

sns.barplot(x='Sex', y='Survived', data=train, ax=axis1)
sns.barplot(x='Pclass', y='Survived', data=train, ax=axis2)
sns.barplot(x='Embarked', y='Survived', data=train, ax=axis3);

In [ ]:
# histograma para ver influência da idade na probabilidade de sobrevivência

age_survived = sns.FacetGrid(train, col='Survived')
age_survived.map(sns.distplot, 'Age');
#age_survived.map(sns.displot, 'Age', kde=True);

In [ ]:
# plota uma scatter matrix

columns = ['Parch', 'SibSp', 'Age', 'Pclass']
pd.plotting.scatter_matrix(train[columns], figsize=(15,10));

In [ ]:
# define função para detectar dados outliers
# retorna lista com os registros outliers de acordo com o método de Tukey

from collections import Counter

def detect_outliers(df,n,features):

    outlier_indices = []
    
    # itera sobre as colunas
    for col in features:
        # 1o quartil (25%)
        Q1 = np.percentile(df[col], 25)
        # 3o quartil (75%)
        Q3 = np.percentile(df[col],75)
        # intervalo interquartil (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # cria lista de índices de outliers por coluna
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # insere o outlier da coluna a uma lista de outliers
        outlier_indices.extend(outlier_list_col)
        
    # seleciona linhas contendo mais de 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detecta outliers de Age, SibSp, Parch e Fare
outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])

# mostra as linhas com valores outliers
train.loc[outliers_to_drop]

In [ ]:
# plota heatmap de correlação entre dados númericos

plt.figure(figsize=(14,12))
plt.title('Correlações entre variáveis', y=1.05, size=15)
sns.heatmap(train.corr(), cmap='coolwarm', fmt='.2f', linewidths=0.1, vmax=1.0, square=True, linecolor='white', annot=True);

## Transformação dos dados

In [ ]:
# concatena dados em um único dataframe e preenche campos vazios e nulos com NaN
df_total = pd.concat(objs=[train, test], axis=0).reset_index(drop=True)
df_total = df_total.fillna(np.nan)

# verifica quantidade de valores nulos no conjunto inteiro
df_total.isnull().sum()

In [ ]:
# cria lista para o total de dados

data = [train, test]

In [ ]:
# cria coluna que informa se um passageiro possuía uma cabine

train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

train.head()

In [ ]:
# cria coluna que informa o tamanho da família do passageiro (contando com ele, combinando SibSp e Parch)

train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

train.head()

In [ ]:
# remove todos os valores NULL na coluna Embarked, preenchendo com o valor modal

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
# remove todos os valores NULL na coluna Fare, preenchendo com o valor mediano

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

In [ ]:
# cria coluna com categorias (classes) de tarifas (Fare), dividindo os dados de treino em quartis

train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

train.head()

In [ ]:
# remove todos os valores NULL na coluna Age, preenchendo com valores aleatórios dentro do desvio-padrão considerando a média de idades

for dataset in data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

In [ ]:
# cria coluna com categorias (classes) de idades (Age), dividindo os dados de treino em quintis

train['CategoricalAge'] = pd.cut(train['Age'], 5)

train.head()

In [ ]:
# define função para extrair o título dos nomes dos passageiros

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    
    # se existir o título, extrai e retorna
    if title_search:
        return title_search.group(1)
    return ""

In [ ]:
# cria coluna contendo o título dos nomes dos passageiros

for dataset in data:
    dataset['Title'] = dataset['Name'].apply(get_title)

train.head()

In [ ]:
# conversão dos dados 

for dataset in data:
    
    # agrupa títulos não comuns em um único grupo "Rare"
    
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
    # converte Sex em 0 ou 1
    
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # converte títulos em 1, 2, 3, 4 ou 5
    
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # converte Embarked em 0, 1 ou 2
    
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # agrupa valores de Fare e converte em 0, 1, 2 ou 3
    
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # agrupa valores de Age e converte em 0, 1, 2, 3 ou 4
    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4;

train.head()

In [ ]:
# exclui colunas desnecessárias

drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [ ]:
# plota heatmap de correlação entre dados númericos

plt.figure(figsize=(14,12))
plt.title('Correlações entre variáveis', y=1.05, size=15)
sns.heatmap(train.corr(), cmap='coolwarm', fmt='.2f', linewidths=0.1, vmax=1.0, square=True, linecolor='white', annot=True);

In [ ]:
# retorna ndarray (array multidimensional) da coluna Survived do conjunto de treino

y_train = train['Survived'].ravel()

In [ ]:
# exclui a coluna Survived do dataset de treino

train.drop(['Survived'], axis=1, inplace=True)

In [ ]:
# cria arrays com os dados de teste e com os dados de treino

x_train = train.values
x_test = test.values

## Implementando modelos de Machine Learning

### Regressão Logística (Logistic Regression)

In [ ]:
# importa biblioteca de Regressão Logística

from sklearn.linear_model import LogisticRegression

# cria modelo de Regressão Logística

lr_model = LogisticRegression(solver='liblinear')
lr_model.fit(x_train, y_train)
lr_predictions = lr_model.predict(x_test);

#### Avaliação do modelo de Regressão Logística

In [ ]:
# analisa a acurácia do modelo de Regressão Logística

acc_logReg = round(lr_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo de Regressão Logística: {}".format(acc_logReg))

### Árvore de Decisão (Decision Tree)

In [ ]:
# importa biblioteca de Árvore de Decisão

from sklearn.tree import DecisionTreeClassifier

# cria modelo de Árvore de Decisão

tree_model = DecisionTreeClassifier(max_depth=3)
tree_model.fit(x_train, y_train)
tree_predictions = tree_model.predict(x_test);

#### Avaliação do modelo de Árvore de Decisão

In [ ]:
# analisa a acurácia do modelo de Árvore de Decisão

acc_tree = round(tree_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo de Regressão Logística: {}".format(acc_tree))

### Bagging

In [ ]:
# importa biblioteca de Bagging

from sklearn.ensemble import BaggingClassifier

# cria modelo Bagging

bg_model = BaggingClassifier()
bg_model.fit(x_train, y_train)
bg_predictions = bg_model.predict(x_test);

#### Avaliação do modelo Bagging

In [ ]:
# analisa a acurácia do modelo Bagging

acc_bg = round(bg_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo Bagging: {}".format(acc_bg))

### Random Forest

In [ ]:
# importa biblioteca de Random Forest

from sklearn.ensemble import RandomForestClassifier

# cria modelo de Random Forest

rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
rf_predictions = rf_model.predict(x_test);

#### Avaliação do modelo de Random Forest

In [ ]:
# analisa a acurácia do modelo de Random Forest

acc_rf = round(rf_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo de Random Forest: {}".format(acc_rf))

### AdaBoost 

In [ ]:
# importa biblioteca de AdaBoost

from sklearn.ensemble import AdaBoostClassifier

# cria modelo de AdaBoost

bst_model = AdaBoostClassifier(n_estimators=100)
bst_model.fit(x_train, y_train)
bst_predictions = bst_model.predict(x_test);

#### Avaliação do modelo AdaBoost

In [ ]:
# analisa a acurácia do modelo de AdaBoost

acc_bst = round(bst_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo de AdaBoost: {}".format(acc_bst))

### Gradient Boosting

In [ ]:
# importa biblioteca de Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier

# cria modelo de Gradient Boosting

gbst_model = GradientBoostingClassifier(n_estimators=100)
gbst_model.fit(x_train, y_train)
gbst_predictions = gbst_model.predict(x_test);

#### Avaliação do modelo de Gradient Boosting

In [ ]:
# analisa a acurácia do modelo de Gradient Boosting

acc_gbst = round(gbst_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo de Gradient Boosting: {}".format(acc_gbst))

### XGB

In [ ]:
# importa biblioteca de XGB

import xgboost as xgb

# cria modelo XGB

xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)
xgb_predictions = xgb_model.predict(x_test);

#### Avaliação do modelo XGB

In [ ]:
# analisa a acurácia do modelo XGB

acc_xgb = round(xgb_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo XGB: {}".format(acc_xgb))

### SVC (Support Vector Machines)

In [ ]:
# importa biblioteca de SVC

from sklearn.svm import SVC

# cria modelo SVC

svc_model = SVC()
svc_model.fit(x_train, y_train)
svc_predictions = svc_model.predict(x_test);

#### Avaliação do modelo SVC

In [ ]:
# analisa a acurácia do modelo SVC

acc_svc = round(svc_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo SVC: {}".format(acc_svc))

### Linear SVC

In [ ]:
# importa biblioteca de LinearSVC

from sklearn.svm import LinearSVC

# cria modelo Linear SVC

lsvc_model = LinearSVC()
lsvc_model.fit(x_train, y_train)
lsvc_predictions = lsvc_model.predict(x_test);

#### Avaliação do modelo Linear SVC

In [ ]:
# analisa a acurácia do modelo Linear SVC

acc_lsvc = round(lsvc_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo Linear SVC: {}".format(acc_lsvc))

### KNN (K-Nearest Neighbors)

In [ ]:
# importa biblioteca de KNN

from sklearn.neighbors import KNeighborsClassifier

# cria modelo KNN

knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)
knn_predictions = knn_model.predict(x_test);

#### Avaliação do modelo KNN

In [ ]:
# analisa a acurácia do modelo KNN

acc_knn = round(knn_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo KNN: {}".format(acc_knn))

### Gaussian Naive Bayes

In [ ]:
# importa biblioteca de Gaussian Naive Bayes

from sklearn.naive_bayes import GaussianNB

# cria modelo Gaussian Naive Bayes

gnb_model = GaussianNB()
gnb_model.fit(x_train, y_train)
gnb_predictions = gnb_model.predict(x_test);

#### Avaliação do modelo Gaussian Naive Bayes

In [ ]:
# analisa a acurácia do modelo Gaussian Naive Bayes

acc_gnb = round(gnb_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo Gaussian Naive Bayes: {}".format(acc_gnb))

### Perceptron

In [ ]:
# importa biblioteca de Perceptron

from sklearn.linear_model import Perceptron

# cria modelo Perceptron

percept_model = Perceptron()
percept_model.fit(x_train, y_train)
percept_predictions = percept_model.predict(x_test);

#### Avaliação do modelo Perceptron

In [ ]:
# analisa a acurácia do modelo Perceptron

acc_percept = round(percept_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo Perceptron: {}".format(acc_percept))

### Stochastic Gradient Descent

In [ ]:
# importa biblioteca de Stochastic Gradient Descent

from sklearn.linear_model import SGDClassifier

# cria modelo Stochastic Gradient Descent

sgd_model = SGDClassifier()
sgd_model.fit(x_train, y_train)
sgd_predictions = sgd_model.predict(x_test);

#### Avaliação do modelo Perceptron

In [ ]:
# analisa a acurácia do modelo Stochastic Gradient Descent

acc_sgd = round(sgd_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo Stochastic Gradient Descent: {}".format(acc_sgd))

### Extra Trees Classifier

In [ ]:
# importa biblioteca de Extra Trees Classifier

from sklearn.ensemble import ExtraTreesClassifier

# cria modelo Extra Trees Classifier

etrees_model = ExtraTreesClassifier()
etrees_model.fit(x_train, y_train)
etrees_predictions = etrees_model.predict(x_test);

#### Avaliação do modelo Extra Trees Classifier

In [ ]:
# analisa a acurácia do modelo Extra Trees Classifier

acc_etrees = round(etrees_model.score(x_train, y_train) * 100, 2)
print("Acurácia do modelo Extra Trees Classifier: {}".format(acc_etrees))

## Verificação do melhor modelo de Machine Learning para o caso

In [ ]:
# lista acurácia dos modelos em sua configuração padrão, em ordem decrescente

models = pd.DataFrame({
    'Modelo': ['Regressão Logística (Logistic Regression)', 'Árvore de Decisão (Decision Tree)',
              'Bagging', 'Random Forest', 'AdaBoost', 'Gradient Boosting',
              'XGB', 'SVC', 'Linear SVC', 'KNN', 'Gaussian Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent',
              'Extra Trees'],
    'Score': [acc_logReg, acc_tree,
              acc_bg, acc_rf, acc_bst, acc_gbst,
              acc_xgb, acc_svc, acc_lsvc, acc_knn, acc_gnb, acc_percept, acc_sgd, acc_etrees]})

models.sort_values(by='Score', ascending=False)

In [ ]:
# testa diversos hiperparâmetros para encontrar o que pode ser mais eficiente no modelo selecionado

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint

model_params = {
    # amostra aleatória de 4 a 200 estimators
    'n_estimators': randint(4,200),
    # max_features normalmente distribuída, com média .25, desvio-padrão 0.1, limitado entre 0 e 1
    'max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
    # distribuição uniforme de 0.01 a 0.2 (0.01 + 0.199)
    'min_samples_split': uniform(0.01, 0.199)
}

rf_model = RandomForestClassifier() # incluir aqui o modelo para o teste

# configura a busca aleatória de metaestimadores
# isto irá treinar 100 modelos através de 5 dobras de cross validation (500 modelos no total)
clf = RandomizedSearchCV(rf_model, model_params, n_iter=100, cv=5, random_state=1)

# treina os modelos para encontrar o melhor modelo em cada 100 candidatos
model = clf.fit(x_train, y_train)

# imprime o conjunto vencedor de hiperparâmetros
from pprint import pprint
pprint(model.best_estimator_.get_params())

## Avaliações Adicionais

In [ ]:
# cria matriz de confusão

# importa bibliotecas de Cross Validation e Confusion Matrix

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# a primeira linha da matriz é sobre as predições dos não sobreviventes:
# em [0,0] temos passageiros que foram classificados corretamente como não sobreviventes (chamados de verdadeiros negativos)
# em [0,1] temos passageiros que foram erroneamente classificados como não sobreviventes (falsos negativos)

# a segunda linha da matriz é sobre as previsões dos sobreviventes:
# em [1,0] temos passageiros classificados erroneamente como sobreviventes (falsos positivos)
# em [1,1] temos passageiros corretamente classificados como sobreviventes (positivos verdadeiros)

predictions = cross_val_predict(rf_model, x_train, y_train, cv=3)
confusion_matrix(y_train, predictions)

In [ ]:
# mede a precisão e a recuperação do modelo

# importa bibliotecas de Precision Score e Recall Score

from sklearn.metrics import precision_score, recall_score

print("Precisão (Precision):", precision_score(y_train, predictions))
print("Recuperação (Recall):",recall_score(y_train, predictions))

In [ ]:
# importa bibliotecas de curva de Precision e Recall

from sklearn.metrics import precision_recall_curve

# Obtendo as probabilidades de nossas previsões

y_scores = rf_model.predict_proba(x_train)
y_scores = y_scores[:,1]

precision, recall, threshold = precision_recall_curve(y_train, y_scores)

# plota gráfico de precisão e recuperação

def plot_precision_and_recall(precision, recall, threshold):
    plt.plot(threshold, precision[:-1], "r-", label="Precisão (precision)", linewidth=5)
    plt.plot(threshold, recall[:-1], "b", label="Recuperação (recall)", linewidth=5)
    plt.xlabel("Limite", fontsize=19)
    plt.legend(loc="upper right", fontsize=19)
    plt.ylim([0, 1])

plt.figure(figsize=(17, 9))
plot_precision_and_recall(precision, recall, threshold)
plt.show()

In [ ]:
# plota gráfico Recall x Precision

def plot_precision_vs_recall(precision, recall):
    plt.plot(recall, precision, "g--", linewidth=2.5)
    plt.ylabel("Recuperação (Recall)", fontsize=19)
    plt.xlabel("Precisão (Precision)", fontsize=19)
    plt.axis([0, 1.5, 0, 1.5])

plt.figure(figsize=(14, 7))
plot_precision_vs_recall(precision, recall)
plt.show()

In [ ]:
# importa biblioteca de curva ROC

from sklearn.metrics import roc_curve

# calcular taxa positiva verdadeira e taxa de falsos positivos

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, y_scores)

# plotando a Curva ROC AUC

def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('Taxa de Falsos Positivos', fontsize=16)
    plt.ylabel('Taxa de Verdaeiros Positivos', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

In [ ]:
# mede a ROC AUC

# importa biblioteca de ROC AUC Score

from sklearn.metrics import roc_auc_score

# cria a medida ROC AUC

r_a_score = roc_auc_score(y_train, y_scores)

print("ROC-AUC Score:", r_a_score)

## Previsão de um cenário específico

In [ ]:
# prevê um cenário específico
# sobrevive? => 1 = Sim; 0 = Não

# array = Pclass, Sex, Age, Parch, Fare, Embarked, Has_Cabin, FamilySize, Title

# Pclass =>      1a Classe = 1; 2a Classe = 2; 3a Classe = 3
# Sex =>         Masculino = 1; Feminino = 0
# Age =>         <= 16 = 0; > 16 <= 32 = 1; > 32 <= 48 = 2; > 48 <= 64 = 3; > 64 = 4
# Parch =>       Quantidade de pais e/ou filhos
# Fare =>        <= 7.91 = 0; > 7.91 <= 14.454 = 1; > 14.454 <= 31 = 2; > 31 = 3
# Embarked =>    Southampton = 0; Cherbourg-Octeville = 1; Queenstown = 2
# Has_Cabin =>   Possui cabine = 1; Não possui cabine = 0
# FamilySize =>  Tamanho do grupo familiar, contando com o passageiro
# Title =>       Mr. = 1; Miss = 2; Mrs. = 3, Master = 4, Outros = 5

rose = np.array([2,0,1,0,1,0,1,2,2]).reshape(1, -1)
jack =  np.array([2,1,1,0,1,0,1,2,1]).reshape(1, -1)

print("Rose sobrevive?\t{}".format(rf_model.predict(rose)[0]))
print("Jack sobrevive?\t{}".format(rf_model.predict(jack)[0]))

## Gerando arquivo para envio ao Kaggle

In [ ]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId, 'Survived': rf_predictions })

In [ ]:
# gerando arquivo csv para envio
StackingSubmission.to_csv("rf_submission.csv", index=False)

StackingSubmission.head()